# SageMaker SDK를 활용하여 서빙하기. (a.k.a Bring Your Own Model)
<img src = "deploy-sagemaker.jpg">


### 대상
- 로컬에서 직접하는 모델을 sagemaker를 활용하여 서빙만 하고 싶은 경우.
- api 서버 개발은 하지 않고 오직 모델만 서빙하고 싶은 경우.

## 1. Model Artifact를 만들기.
- 모델 추론을 위한 코드와 관련 Asset들을 포함한 압축파일.
- 모델 artifact는 실제 deploy를 할 때, aws에서 제공하는 도커 환경에 decompressed되는 파일.
    - **정해진 경로와 네이밍을 잘 맞추는 것이 중요함.**

### 1.1. 모델 artifact의 구조
```bash
# tar file을 만들 때, 실제 model 경로 안에 들어가서 `tar -zcvf model.tar.gz .`을 실행해야 됨.

model.tar.gz/
|- model.bin # model binary.
|- ... # assets.
|- code/
  |- inference.py # inference code,
  |- requirements.txt  # python packages need to be installed.
```

**필수**
- model.bin (모델 바이너리 파일)
- inference.py (모델 추론 관련 코드)

**optional**
- config.json
- vocab.txt (토크나이저)
- etc...

### 1.2. `inference.py`를 정의하기.
- `inference.py`은 아래의 네 가지 function 정의해야 됨.
    - `model_fn`
        - 추론에 사용할 모델을 로드하는 부분.
        - return을 tuple 또는 dictionary로 받으면 여러개의 return을 받을 수 있다.
    - `input_fn`: request data를 deserialize하고, prediction을 위한 input 형태로 변환하는 부분.
        - 다양한 request_content_type에 따라서 request body가 어떻게 들어오는지 확인 필요함.
    - `predict_fn`: 실제 inference 코드를 수행하는 부분.
    - `output_fn`: result of prediction을 serialize하여, 리턴하는 부분
    - 함수의 input과 output은 다음과 같은 flow로 흘러감.
        - (model_fn, input_fn) $\longrightarrow$ predict_fn $\longrightarrow$ output_fn
- input, ouput의 가장 간단한 방식은 json으로 데이터를 주고 받는 것.

## 2. Deploy Model with SageMaker SDK


In [1]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

- role과 필요한 변수 정의

In [2]:
role = get_execution_role()
model_artifact_path = "s3://kdw-sagemaker/model/pytorch2/model.tar.gz"
instance_type = "ml.g4dn.xlarge"

- PyTorchModel SageMaker SDK 정의하기

In [3]:
model = PyTorchModel(
    entry_point="inference.py", # inference.py의 파일명.
    role=role, # role
    model_data=model_artifact_path, # model_artifact의 경로
    framework_version="1.8.1", # pytorch version
    py_version="py3" # python version
)

- model deploy하기

In [20]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    endpoint_name="naver-ner"
)

-------------------!

## 3. Call the Endpoint

### 3-1. SageMaker SDK 사용하기
- model deploy가 된 객체가 있어야됨.

In [22]:
%%time
dummy_data = {'text': '아모레퍼시픽은 화장품 회사다.'}
res = predictor.predict(dummy_data)
print("Predictions:", res)

Predictions: {'predicts': [['O', 'ORG-B', 'ORG-B', 'ORG-B', 'ORG-B', 'ORG-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]}
CPU times: user 0 ns, sys: 3.32 ms, total: 3.32 ms
Wall time: 36.3 ms


### 3-2. Boto3로 call 하기.

In [23]:
import io
import json
import boto3

In [24]:
runtime= boto3.client('runtime.sagemaker')
endpoint_name = "naver-ner"
payload = json.dumps({'text': '아모레퍼시픽은 화장품 회사다.'})

In [29]:
%%time
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                       ContentType='application/json',
                                       Body=payload)

print(json.loads(response['Body'].read().decode()))

{'predicts': [['O', 'ORG-B', 'ORG-B', 'ORG-B', 'ORG-B', 'ORG-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]}
CPU times: user 3.13 ms, sys: 0 ns, total: 3.13 ms
Wall time: 22.4 ms
